In [6]:
import statsmodels
import seaborn as sns
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats



def compute_auc(array_1, array_2):
    '''Compute AUROC.'''
    xs = np.concatenate([array_1, array_2],axis=1)
    y = np.concatenate([array_1.shape[1]*[2], array_2.shape[1]*[1]])

    for i, x in enumerate(xs):
        fpr, tpr, thresholds = metrics.roc_curve(y, x, pos_label=2)
        #print(i, metrics.auc(fpr, tpr))
        m = metrics.roc_auc_score(y, x)
        print(round(max(m, 1-m), 2))



def delete_multiple_element(list_object, indices):
    indices = sorted(indices, reverse=True)
    for idx in indices:
        if idx < len(list_object):
            list_object.pop(idx)
            
    return list_object



def kruskal(f, task, c, p, c_name, p_name):
    for i, title in enumerate(task):
        nome = title
        f.write(('\n'+ f'kruskal results for {title} {c_name} {p_name} {stats.kruskal(c[i], p[i])} \n\n'))

def holm_correction(kruskal):
    line_to_remove=[]
    values=[]
    corrected =[]
    final = []
    for l in kruskal:
        if "nan" in l:
            line_to_remove.append(kruskal.index(l))
    
    new_krusk = delete_multiple_element(kruskal, line_to_remove)
            
    for line in new_krusk:
        ok = line.split('vs.')[1]
        num = ok.split(" ")[2]
        values.append(float(num))
   # values = [x for x in values if isnan(x) == False]
    result = statsmodels.stats.multitest.fdrcorrection(values, alpha=0.05, method='indep', is_sorted=False)
    num = np.where(result[0] == True)
    list_index = ((num)[0]).tolist()

    for i in list_index:
        corrected.append(result[1][i])
    for i in list_index:
        final.append(kruskal[i])
    
    return final, corrected
    
#Speech transcripts Cookie Theft picture description task

input_dir = '/export/c12/afavaro/Transcriptions/CTP/'


path = []
for ok in os.listdir(input_dir):
    if ok.endswith('.txt'):
        path.append(os.path.join(input_dir, ok))
len(path)   

transcr = []
for file in path:
    with open(file, "r") as f:
        string_without_line_breaks = ""
        for line in f:
            stripped_line = line.rstrip()
            string_without_line_breaks += stripped_line
        transcr.append(string_without_line_breaks)

speaker = [os.path.basename(file).split("_ses")[0] for file in path]
dataframe = {'idx':speaker, 'sentence': transcr}
df_10msec = pd.DataFrame(dataframe)
pd.set_option('display.max_colwidth', None)

# Add labels

label_seneca = pd.read_excel("/export/b14/afavaro/Acoustic_Features/DigiPsych_Prosody/Hospital_Analysis/Book3.xlsx")
label = label_seneca['Label'].tolist()
speak = label_seneca['Participant I.D.'].tolist() #id
spk2lab_ = {sp:lab for sp,lab in zip(speak,label)}
speak2__ = df_10msec['idx'].tolist()


etichettex = []
for nome in speak2__:
    if nome in spk2lab_.keys():
        lav = spk2lab_[nome]
        etichettex.append(([nome, lav]))
    else:
         etichettex.append(([nome, 'Unknown']))



label_new_ = []
for e in etichettex:
    label_new_.append(e[1])
df_10msec['label'] = label_new_



def uncertanty(text):
    
    ''' Function design to capture the level of certainty of patients of participants when delivering  the description
    of the image.  '''
    
    cont_con = 0
    if "?" in text:
        cont_con = cont_con + 1
    if "why" in text:
        cont_con = cont_con + 1
    if "might" in text:
        cont_con = cont_con + 1
    #if "can" in text:
      #  cont_con = cont_con + 1
    if "could" in text:
        cont_con = cont_con + 1
    if "may" in text:
        cont_con = cont_con + 1
    #if "not sure" in text:
       # cont_con = cont_con + 1     
   # if "I" in text:
      #  cont_con = cont_con + 1 
    if "um" in text:
        cont_con = cont_con + 1 
    if "uhm" in text:
        cont_con = cont_con + 1 
    if "ah" in text:
        cont_con = cont_con + 1 
    if "perhaps" in text:
        cont_con = cont_con + 1 
    #if "should" in text:
        #cont_con = cont_con + 1 
    if "looks like" in text:
        cont_con = cont_con + 1
        
    
    return cont_con

df['uncertanty'] = df['sentence'].apply(uncertanty)

In [19]:

def repetitions(text):
     
    '''Function design to capture the redundancy in the code.  To operationalize 
    redundancy I chose to count the repetitions. '''
    
    
    repetition = 0
    
    text = text.split()
    d = dict()
    
    for line in text:
        line = line.strip()
        line = line.lower()
        words = line.split(" ")
        for word in words:

            if word in d:
                d[word] = d[word] + 1
            else:
                d[word] = 1
    
    for key in list(d.keys()):
        if key != 'the' and key != "a" and key != "of" and key != "to" and key !="on" and key !="is" and key !="are" and key !="in" and key != "an":
            if d[key] > 1:
                repetition +=1

    return repetition


df['repetition'] = df['sentence'].apply(repetitions)


In [20]:
def informational_verb(text):
    
    '''Exhaustivness of the description operationalized by 
    counting how many (if any) salient items (nouns) are mentioned. '''
    
    cont_con = 0
    
    if "washing" in text:
        cont_con = cont_con + 1
    if "wash" in text:
        cont_con = cont_con + 1
    
    if "overflowing" in text:
        cont_con = cont_con + 1
      
    if "overflow" in text:
        cont_con = cont_con + 1
        
    if "hanging" in text:
        cont_con = cont_con + 1
    if "hang" in text:
        cont_con = cont_con + 1
        
    if "falling" in text:
        cont_con = cont_con + 1
    if "fall" in text:
        cont_con = cont_con + 1
        
    if "wearing" in text:
        cont_con = cont_con + 1
    if "wear" in text:
        cont_con = cont_con + 1
    if "running" in text:
        cont_con = cont_con + 1
        
    if "run" in text:
        cont_con = cont_con + 1
        
    if "drying" in text:
        cont_con = cont_con + 1
    if "dry" in text:
        cont_con = cont_con + 1
    if "paying attention" in text:
        cont_con = cont_con + 1
    if "reaching" in text:
        cont_con = cont_con + 1
    if "reach" in text:
        cont_con = cont_con + 1
    if "tipping" in text:
        cont_con = cont_con + 1
    if "tipp" in text:
        cont_con = cont_con + 1


    
    return cont_con

df['informational_verb'] = df['sentence'].apply(informational_verb)

In [21]:
def informational_content(text):

    '''Exhaustivness of the description operationalized by 
    counting how many (if any) salient actions (verbs) are mentioned. '''

    
    cont_con = 0
    
    if "mother" in text:
        cont_con = cont_con + 1
    if "sister" in text:
        cont_con = cont_con + 1
    if "cookie" in text:
        cont_con = cont_con + 1
    if "cookie jar" in text:
        cont_con = cont_con + 1
    if "curtains" in text:
        cont_con = cont_con + 1
    if "cabinet" in text:
        cont_con = cont_con + 1
    if "brother" in text:
        cont_con = cont_con + 1
    if "chair" in text:
        cont_con = cont_con + 1
    if "kitchen" in text:
        cont_con = cont_con + 1
    if "sink" in text:
        cont_con = cont_con + 1
    if "garden" in text:
        cont_con = cont_con + 1
    if "fall" in text:
        cont_con = cont_con + 1
    if "dishes" in text:
        cont_con = cont_con + 1     
    if "stool" in text:
        cont_con = cont_con + 1 
    if "poddle" in text:
        cont_con = cont_con + 1 
    if "shoes" in text:
        cont_con = cont_con + 1 
    if "apron" in text:
        cont_con = cont_con + 1 
    
    return cont_con

df['informational'] = df['sentence'].apply(informational_content)


def ratio_info_rep_plus_uncert(df_): 
    
    '''Ratio between repetitions and uncertanty,
    where uncertainty is operationalized as repetition + uncertanty. '''
    
    summation = df_['repetition'] + df_["uncertanty"]
    ratio = df_['informational'] / summation #info / rep + uncertanty
    df_["ratio_info_rep_plus_uncert"] = ratio
    
    return df_

df = ratio_info_rep_plus_uncert(df)


def ratio_rep_certanty(df_): 
    
    '''Function designed to measure the ratio between repetitions and uncertanty,
    where uncertainty is intended as uncertainty. '''
    
    division = df_['repetition'] / df_["uncertanty"] # repetition / uncertainty
    df_["ratio_rep_certanty"] = division
    
    return df_

df = ratio_rep_certanty(df)



grouped = df.groupby('label')


control = grouped.get_group("CTRL") 
parkinson_ = grouped.get_group("PD")
alzheimer =  grouped.get_group("AD")
#ataxia =  grouped.get_group("ATX")
others =  pd.concat([ grouped.get_group('CBS'),  
                     grouped.get_group('ET'), grouped.get_group('GSS'), grouped.get_group('WD'), 
                      grouped.get_group('PSP'), grouped.get_group('MIM')])



pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


# Remve probable PD subjects

parkinson_ = parkinson_[~parkinson_.idx.str.contains("NLS_102")]
parkinson_ = parkinson_[~parkinson_.idx.str.contains("NLS_98")]
parkinson_ = parkinson_[~parkinson_.idx.str.contains("NLS_95")]
parkinson_ = parkinson_[~parkinson_.idx.str.contains("NLS_87")]
parkinson_ = parkinson_[~parkinson_.idx.str.contains("NLS_85")]
parkinson_ = parkinson_[~parkinson_.idx.str.contains("NLS_57")]
parkinson_ = parkinson_[~parkinson_.idx.str.contains("NLS_34")]
parkinson_ = parkinson_[~parkinson_.idx.str.contains("NLS_33")]
parkinson_ = parkinson_[~parkinson_.idx.str.contains("NLS_12")]
parkinson_ = parkinson_[~parkinson_.idx.str.contains("NLS_21")]


In [22]:
task = df.columns[3:].values.tolist()

alzheimer_all_feat = np.array([alzheimer[feat] for feat in np.array(df.columns[3:])])
others_all_feat = np.array([others[feat] for feat in np.array(df.columns[3:])])
parkinson_all_feat = np.array([parkinson_[feat] for feat in np.array(df.columns[3:])])
control_all_feat =np.array([control[feat] for feat in np.array(df.columns[3:])])
#ataxia_all_feat =np.array([ataxia[feat] for feat in np.array(df.columns[3:])])


with open('/export/b14/afavaro/SLT_submission/cognitive/results/cn_vs_pd.txt', 'w') as f:
 
    kruskal(f, task, control_all_feat, parkinson_all_feat, "controls vs.", "parkinson")

In [75]:
with open('/export/b14/afavaro/SLT_submission/cognitive/results/cn_vs_otr.txt', 'w') as f:
    
    
    kruskal(f, task, control_all_feat, others_all_feat, "control vs.", "others")
    

In [76]:
with open('/export/b14/afavaro/SLT_submission/cognitive/results/pd_vs_otr.txt', 'w') as f:
    
    
    kruskal(f, task, parkinson_all_feat, others_all_feat, "parkinson vs.", "others")
    

In [77]:
with open('/export/b14/afavaro/SLT_submission/cognitive/results/ad_vs_otr.txt', 'w') as f:

    
    kruskal(f, task, alzheimer_all_feat, others_all_feat, "alzheimers vs.", "others")
    

In [78]:
with open('/export/b14/afavaro/SLT_submission/cognitive/results/ad_vs_cn.txt', 'w') as f:
    #final_sub
    
    kruskal(f, task, alzheimer_all_feat, control_all_feat, "alzheimers vs.", "controls")
    

In [79]:
with open('/export/b14/afavaro/SLT_submission/cognitive/results/ad_vs_pd.txt', 'w') as f:
    #final_sub
    
    kruskal(f, task, alzheimer_all_feat, parkinson_all_feat, "alzheimers vs.", "parkinson")


kruskal_1 = read_stats_test('/export/b14/afavaro/SLT_submission/cognitive/results/ad_vs_cn.txt')

kruskal_1_ = read_stats_test('/export/b14/afavaro/SLT_submission/cognitive/results/ry_ad_vs_cn.txt')
kruskal_1__= read_stats_test('/export/b14/afavaro/SLT_submission/cognitive/results/rt_ad_vs_cn.txt')
kruskal_2 = read_stats_test('/export/b14/afavaro/SLT_submission/cognitive/results/ad_vs_otr.txt')
kruskal_3 = read_stats_test('/export/b14/afavaro/SLT_submission/cognitive/results/ad_vs_pd.txt')
kruskal_4 = read_stats_test('/export/b14/afavaro/SLT_submission/cognitive/results/cn_vs_otr.txt')
kruskal_4_ = read_stats_test('/export/b14/afavaro/SLT_submission/cognitive/results/ry_cn_vs_otr.txt')
kruskal_4__ = read_stats_test('/export/b14/afavaro/SLT_submission/cognitive/results/ry_cn_vs_otr.txt')
kruskal_4__ = read_stats_test('/export/b14/afavaro/SLT_submission/cognitive/results/rt_cn_vs_otr.txt')
kruskal_5 = read_stats_test('/export/b14/afavaro/SLT_submission/cognitive/results/cn_vs_pd.txt')
kruskal_5_ = read_stats_test('/export/b14/afavaro/SLT_submission/cognitive/results/rt_cn_vs_pd.txt')
kruskal_6 = read_stats_test('/export/b14/afavaro/SLT_submission/cognitive/results/pd_vs_otr.txt')
#best_krusk = compute_best_scores(kruskal)


#PLOT DISTRIBUTIONS

for i, title in enumerate(task):  
    
    nome = title
   
    controls = []
    parkinson=[]
    alz = []
    #ataxia = []
    others= []
    
    data = np.concatenate([control_all_feat[i], parkinson_all_feat[i], alzheimer_all_feat[i], others_all_feat[i]])
    data = data.tolist()

    [controls.append('CN') for value in range(len(control_all_feat[i]))]
    #[ataxia.append('ATX') for value in range(len(ataxia_all_feat[i]))]
    [parkinson.append('PD') for value in range(len(parkinson_all_feat[i]))]
    [alz.append('AD') for value in range(len(alzheimer_all_feat[i]))]
    [others.append('PDM') for value in range(len(others_all_feat[i]))]

    lista = controls+parkinson+alz+others
    
    dict = {nome: data, 'Group': lista} 
    df = pd.DataFrame(dict)

    sns.set(font_scale=2)
    sns.catplot(x=nome, y="Group", kind="boxen", data=df, k_depth='full', palette="Blues",\
               height=4.2, aspect=1.4, showfliers = False)
    
    plt.savefig(f'/export/b14/afavaro/SLT_submission/Final_plot_SLT/{nome}_', facecolor='white', dpi=600)



